In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np




In [ ]:
with open("alchemist.txt", 'r', encoding='utf-8') as f:
    text = f.read()

print(f"Total characters: {len(text)}")

Total characters: 234264


In [ ]:
chars = sorted(set(text))
print(len(chars))
char2int = {ch: i for i, ch in enumerate(chars)}
int2char = {i: ch for i, ch in enumerate(chars)}

vocab = len(chars)

84


In [ ]:
chars = sorted(set(text))
char2int = {ch: i for i, ch in enumerate(chars)}
int2char = {i: ch for i, ch in enumerate(chars)}

data_encoded = np.array([char2int[ch] for ch in text], dtype=np.int64)

class TextDataset(Dataset):
  def __init__(self, data, seq_length):
    self.data = data
    self.seq_length = seq_length

  def __len__(self):
    return len(self.data) - self.seq_length


  def __getitem__(self, idx):
    x = self.data[idx: idx+self.seq_length]
    y = self.data[idx+self.seq_length]
    x = torch.tensor(x, dtype=torch.long)
    y = torch.tensor(y, dtype=torch.long)
    return x, y


split = int(len(data_encoded)*0.8)

train_dataset = data_encoded[:split]
test_dataset = data_encoded[split:]

seq_len = 100
batch_size = 64
embed_size = 128
hidden_size = 256
num_layers = 2

train_dataset = TextDataset(train_dataset, seq_len)
test_dataset = TextDataset(test_dataset, seq_len)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)



class CharRNN(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.rnn = nn.RNN(input_size=embed_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size, vocab_size)
    self.hidden_size = hidden_size
    self.num_layers = num_layers

  def forward(self, x, hidden):
    emb = self.embedding(x) # [batch, seq] -> [batch, seq, embed_dim]
    out, hidden = self.rnn(emb, hidden) # out: [batch, seq, hidden]
    last = out[:, -1, :] # [batch, hidden]
    logits = self.fc(last)
    return logits, hidden

  def init_hidden(self, batch_size):
    return torch.zeros(self.num_layers, batch_size, self.hidden_size)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CharRNN(vocab_size=vocab, embed_size=embed_size, hidden_size=hidden_size, num_layers=num_layers).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epochs = 50

model.train()
for epoch in range(1, epochs+1):
  hidden = model.init_hidden(batch_size).to(device)
  total_loss = 0
  for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    optimizer.zero_grad()
    hidden = hidden.detach()
    logits, hidden = model(x_batch, hidden)
    loss = criterion(logits, y_batch)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()

  print(f"Epoch: {epoch}, Loss: {total_loss/len(train_loader)}")




Epoch: 1, Loss: 1.7363349864398137
Epoch: 2, Loss: 1.4310169286175451
Epoch: 3, Loss: 1.3499434082090895
Epoch: 4, Loss: 1.3069081218659837
Epoch: 5, Loss: 1.2812589695212455
Epoch: 6, Loss: 1.261522964997725
Epoch: 7, Loss: 1.246847966254287
Epoch: 8, Loss: 1.2377779281514194
Epoch: 9, Loss: 1.2309103346979138
Epoch: 10, Loss: 1.2254305899306184
Epoch: 11, Loss: 1.223903723290255
Epoch: 12, Loss: 1.2208295333532138
Epoch: 13, Loss: 1.2198815440895292
Epoch: 14, Loss: 1.2233406751023852
Epoch: 15, Loss: 1.2237478244695774
Epoch: 16, Loss: 1.223607191526963
Epoch: 17, Loss: 1.2306933001508797
Epoch: 18, Loss: 1.2270532797764853
Epoch: 19, Loss: 1.2278762900780424
Epoch: 20, Loss: 1.2365187760582543
Epoch: 21, Loss: 1.23674705583839
Epoch: 22, Loss: 1.2406139169664963
Epoch: 23, Loss: 1.2430092484390418
Epoch: 24, Loss: 1.2463172797829178
Epoch: 25, Loss: 1.2476508680753975
Epoch: 26, Loss: 1.2526990157932105
Epoch: 27, Loss: 1.2564218661920388
Epoch: 28, Loss: 1.2645069653459877
Epoch: 

KeyboardInterrupt: 

In [ ]:
# Text generate
def generated_text(model, start_text, length):
  model.eval()
  chars = [ch for ch in start_text]
  input_vector = torch.tensor([char2int[ch] for ch in chars], dtype=torch.long).unsqueeze(0).to(device)
  hidden = model.init_hidden(1).to(device)
  with torch.no_grad():
    for i in range(len(start_text)-1):
      logits, hidden = model(input_vector[:, i:i+1], hidden)
    last_char = input_vector[:, -1]


    for i in range(length):
      logits, hidden = model(last_char.unsqueeze(0), hidden) # logits-> (batch, seq, vocab_size)
      probs = torch.softmax(logits.squeeze(), dim=0)
      probs = probs.cpu().numpy()
      next_idx = np.random.choice(len(probs), p=probs)
      # next_idx = torch.argmax(probs).item()

      next_char = int2char[next_idx]
      chars.append(next_char)
      last_char = torch.tensor([next_idx], dtype=torch.long).to(device)

  return ''.join(chars)


text = 'Maktub,” the boy said, remembering the'
generated = generated_text(model, text, 500)
print("Generated_text: ", generated)

Generated_text:  Maktub,” the boy said, remembering the sentir Gourees, two concerned and maybe things who translated the sact: untill be
 soullowing enemoeled in the boy in lased he had treasured
 asked. He a moment. The ensipe and try alchemy. But the old king a day. The boy
way beginnest when
 the oasis. He said pike, and alrept of the alchemist, and some do the boy
 this it
 had stay
 paying divall traity. The boy had as the boy knew the basis, had being on the vision. He had deal outed palm of every time the awpended that thought there camable 


In [ ]:
probs = [0.1, 0.3, 0.6]
for i in range(10):
  next_idx = np.random.choice(len(probs), p=probs)
  print(next_idx)

2
0
2
2
0
2
2
2
1
1


In [ ]:
x = torch.tensor([[[1,2,3,4,5]]], dtype=torch.float32)
x = x.squeeze()
x
probs = torch.softmax(x, dim=0)
probs

tensor([0.0117, 0.0317, 0.0861, 0.2341, 0.6364])

In [ ]:
input_vector = torch.tensor([1.0,2.0, 3.0])
input_vector.unsqueeze(0)

tensor([[1., 2., 3.]])

In [ ]:
for batch_x, batch_y in train_loader:
  print(batch_x.shape, batch_y.shape)
  break

ValueError: too many values to unpack (expected 2)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
chars

['\n',
 ' ',
 '!',
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '©',
 '®',
 'á',
 'é',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '™']

In [ ]:
embeddings = nn.Embedding(5, 10)
print(embeddings.weight)

input_tokens = torch.tensor([0,1,2])
output = embeddings(input_tokens)
print(output)

Parameter containing:
tensor([[ 0.9895, -0.7138,  1.5428, -0.5540,  0.5835,  0.7710, -0.2709, -0.5589,
          0.1004,  2.6170],
        [-2.2132, -0.8640, -0.7878,  1.1469,  1.7457,  0.1805,  0.2211, -0.3021,
          2.2576,  3.6529],
        [-0.8738, -1.1644,  1.5557,  1.6027,  0.2543, -0.4666,  0.5439, -0.4019,
          1.6052, -0.2464],
        [-0.2778,  0.2834, -0.1060,  1.9152,  0.9526,  0.3398,  0.2784, -0.4145,
          0.1950,  0.5672],
        [ 0.9513, -0.1135,  0.7616,  0.6755, -1.0950, -2.0004,  0.2955, -1.7610,
          0.5836,  1.0470]], requires_grad=True)
tensor([[ 0.9895, -0.7138,  1.5428, -0.5540,  0.5835,  0.7710, -0.2709, -0.5589,
          0.1004,  2.6170],
        [-2.2132, -0.8640, -0.7878,  1.1469,  1.7457,  0.1805,  0.2211, -0.3021,
          2.2576,  3.6529],
        [-0.8738, -1.1644,  1.5557,  1.6027,  0.2543, -0.4666,  0.5439, -0.4019,
          1.6052, -0.2464]], grad_fn=<EmbeddingBackward0>)


'Contents\n International Acclaim for Paulo Coelho’s\n Foreword\n Prologu\n e\n Part One\n Part Two\n Epilogue\n A Preview of Paulo Coelho’s: Warrior of the Light\n Warrior of the Light: Prologue\n About the Author\n Also by Paulo Coelho\n Back Ads\n Copyright\n About the Publisher\nInternational Acclaim for Paulo Coelho’s\n THE ALCHEMIST\n “The story has the comic charm, dramatic tension, and psychological\n intensity of a fairy tale, but it’s full of specific wisdom as well. . . . A\n sweetly exotic tale for youn'

In [ ]:
chars = sorted(set("apple ball cat"))
chars

[' ', 'a', 'b', 'c', 'e', 'l', 'p', 't']

In [ ]:
char2int = {ch: i for i, ch in enumerate(chars)}
char2int

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'e': 4, 'l': 5, 'p': 6, 't': 7}

In [ ]:
text = 'apple ball cat'
index = [char2int[ch] for ch in text]
index

[1, 6, 6, 5, 4, 0, 2, 1, 5, 5, 0, 3, 1, 7]